In [ ]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np
from scipy.stats import norm as scipynorm

pd.set_option('display.float_format','{:.4f}'.format)

hubs_dict = {
    'ARICA': 'IQUIQUE',
    'IQUIQUE': 'IQUIQUE',
    'ANTOFAGASTA': 'ANTOFAGASTA',
    'COPIAPO': 'COPIAPO',
    'COQUIMBO': 'COQUIMBO',
    'OVALLE': 'COQUIMBO',
    'ILLAPEL': 'CURAUMA',
    'LLAY LLAY': 'CURAUMA',
    'CURAUMA': 'CURAUMA',
    'SANTIAGO SUR': 'SANTIAGO SUR',
    'RANCAGUA': 'SANTIAGO SUR',
    'TALCA': 'TALCA',
    'CHILLAN': 'TALCAHUANO',
    'TALCAHUANO': 'TALCAHUANO',
    'LOS ANGELES': 'TALCAHUANO',
    'TEMUCO': 'TEMUCO',
    'VALDIVIA': 'VALDIVIA',
    'OSORNO': 'VALDIVIA',
    'PUERTO MONTT': 'PUERTO MONTT',
    'CASTRO': 'PUERTO MONTT',
    'COYHAIQUE': 'COYHAIQUE',
    'CALAMA': 'ANTOFAGASTA'
}

def stats(datos, confianza, lead_time):
    
    t = scipynorm.ppf(confianza)

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_time, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_time, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: confianza * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

In [2]:
restricted = False
porcentaje = 100
lead_time = 7

In [3]:
datos_limpios = pd.read_csv('../data/datos_limpios_filtrados.csv')
datos_limpios = datos_limpios[(datos_limpios['DESCR_CENDIST'] != 'CERVECERA') & (datos_limpios['DESCR_CENDIST'] != 'MODELO')]
datos_limpios = datos_limpios.drop(columns='Unnamed: 0')

In [32]:
confianza_inicial = 0.95
stats_no_agrupados, porcentaje_ss_no_agrupado = stats(datos_limpios, confianza_inicial, lead_time)

In [33]:
assign_hub = lambda x: hubs_dict[x['DESCR_CENDIST']]
    
datos_para_agrupar = datos_limpios.copy()
datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(assign_hub, axis=1)
datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

-------------------------------------------------------------------------------------------------------------------------------

In [35]:
confianza_final = 0.925
stats_agrupados_925, porcentaje_ss_agrupado_925 = stats(datos_agrupados, confianza_final, lead_time)

In [36]:
confianza_final = 0.95
stats_agrupados_95, porcentaje_ss_agrupado_95 = stats(datos_agrupados, confianza_final, lead_time)

-------------------------------------------------------------------------------------------------------------------------------

In [54]:
finalStocks = stats_agrupados.merge(stats_no_agrupados, on=['ID_SKU_VENTA', 'DESCR_CENDIST'], how='outer')
finalStocks.fillna(0, inplace=True)

In [55]:
agrupados = stats_agrupados_95.merge(stats_agrupados_925, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])
agrupados = agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA_x', 'STD_x', 'SS_x', 'SS_y']]
agrupados.rename(columns={'MEDIA_x': 'MEDIA', 'STD_x': 'STD', 'SS_x': 'SS_95', 'SS_y': 'SS_92.5'}, inplace=True)
# agrupados

In [56]:
no_agrupados = stats_no_agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA', 'STD', 'SS']]
# no_agrupados

In [57]:
todos = no_agrupados.merge(agrupados, on=['ID_SKU_VENTA', 'DESCR_CENDIST'], how='outer').fillna(0)
todos.rename(columns={'MEDIA_x': 'MEDIA_NO_AGR', 'STD_x': 'STD_NO_AGR', 'STD_y': 'STD_AGR', 'SS': 'SS_NO_AGR'}, inplace=True)
todos = todos[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA_NO_AGR', 'STD_NO_AGR', 'STD_AGR', 'SS_NO_AGR', 'SS_95', 'SS_92.5']]
todos

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA_NO_AGR,STD_NO_AGR,STD_AGR,SS_NO_AGR,SS_95,SS_92.5
0,515,ANTOFAGASTA,0.4005,0.7071,1.0041,0.6717,0.9539,0.9288
1,515,ARICA,0.0419,0.1407,0.0000,0.1337,0.0000,0.0000
2,515,CALAMA,0.2196,0.4439,0.0000,0.4218,0.0000,0.0000
3,515,CASTRO,0.1207,0.3007,0.0000,0.2857,0.0000,0.0000
4,515,CHILLAN,0.2766,0.5116,0.0000,0.4860,0.0000,0.0000
...,...,...,...,...,...,...,...,...
23094,870506,IQUIQUE,0.0000,0.0000,0.0027,0.0000,0.0026,0.0025
23095,870614,TALCAHUANO,0.0000,0.0000,0.0255,0.0000,0.0243,0.0236
23096,870871,VALDIVIA,0.0000,0.0000,0.0311,0.0000,0.0296,0.0288
23097,870872,ANTOFAGASTA,0.0000,0.0000,0.0057,0.0000,0.0055,0.0053


In [58]:
todos['NEW_STOCK_CD'] = todos['MEDIA_NO_AGR']
todos['NEW_STOCK_HUB'] = todos['MEDIA_NO_AGR'] + todos['SS_92.5']

In [59]:
hubSet = set(hubs_dict.values())

hubs = todos[todos['DESCR_CENDIST'].isin(hubSet)]
cds = todos[~todos['DESCR_CENDIST'].isin(hubSet)]

In [60]:
hubs['NEW_STOCK'] = hubs['NEW_STOCK_HUB']
cds['NEW_STOCK'] = cds['NEW_STOCK_CD']

cleanHubs = hubs[['ID_SKU_VENTA', 'DESCR_CENDIST', 'NEW_STOCK']]
cleanCds = cds[['ID_SKU_VENTA', 'DESCR_CENDIST', 'NEW_STOCK']]

<ipython-input-60-8d1510776752>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hubs['NEW_STOCK'] = hubs['NEW_STOCK_HUB']
<ipython-input-60-8d1510776752>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cds['NEW_STOCK'] = cds['NEW_STOCK_CD']


In [61]:
newStocks = pd.concat([cleanHubs, cleanCds])
# newStocks

,ID_SKU_VENTA,DESCR_CENDIST,NEW_STOCK
0,515,ANTOFAGASTA,1.3294
5,515,COPIAPO,0.6537
6,515,COQUIMBO,2.3370
7,515,COYHAIQUE,0.1657
9,515,IQUIQUE,0.2587
...,...,...,...
22997,875225,LLAY LLAY,0.0080
22998,875225,LOS ANGELES,0.0076
22999,875225,OSORNO,0.0027
23000,875225,OVALLE,0.0018


In [64]:
cleanStats = stats_no_agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'STOCK_TEORICO']]
cleanStats.rename(columns={'STOCK_TEORICO': 'OLD_STOCK'}, inplace=True)
# cleanStats

c:\users\damen\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,ID_SKU_VENTA,DESCR_CENDIST,OLD_STOCK
0,515,ANTOFAGASTA,1.0722
1,515,ARICA,0.1756
2,515,CALAMA,0.6413
3,515,CASTRO,0.4064
4,515,CHILLAN,0.7626
...,...,...,...
23007,875225,VALDIVIA,0.0042
23008,875359,SANTIAGO SUR,0.2390
23009,875365,CURAUMA,0.3427
23010,875365,SANTIAGO SUR,0.2390


In [71]:
whole = cleanStats.merge(newStocks, on=['ID_SKU_VENTA', 'DESCR_CENDIST'], how='outer').fillna(0)
whole['DELTA'] = whole['NEW_STOCK'] - whole['OLD_STOCK']
whole

,ID_SKU_VENTA,DESCR_CENDIST,OLD_STOCK,NEW_STOCK,DELTA
0,515,ANTOFAGASTA,1.0722,1.3294,0.2571
1,515,ARICA,0.1756,0.0419,-0.1337
2,515,CALAMA,0.6413,0.2196,-0.4218
3,515,CASTRO,0.4064,0.1207,-0.2857
4,515,CHILLAN,0.7626,0.2766,-0.4860
...,...,...,...,...,...
23094,870506,IQUIQUE,0.0000,0.0025,0.0025
23095,870614,TALCAHUANO,0.0000,0.0236,0.0236
23096,870871,VALDIVIA,0.0000,0.0288,0.0288
23097,870872,ANTOFAGASTA,0.0000,0.0053,0.0053


In [72]:
whole.DELTA.sum()

-2310.672916828409

In [73]:
whole.to_excel('tabla agrupacion variabilidad 92.5.xlsx')

In [75]:
wholeAggregated = whole.groupby(by='DESCR_CENDIST').sum().reset_index()
wholeAggregated = wholeAggregated[['DESCR_CENDIST', 'OLD_STOCK', 'NEW_STOCK', 'DELTA']]
wholeAggregated

,DESCR_CENDIST,OLD_STOCK,NEW_STOCK,DELTA
0,ANTOFAGASTA,1743.0223,2152.0860,409.0637
1,ARICA,908.8104,293.7469,-615.0635
2,CALAMA,904.0402,299.2557,-604.7845
3,CASTRO,559.9580,174.2444,-385.7135
4,CHILLAN,1550.6889,516.2376,-1034.4513
5,COPIAPO,1292.3201,1269.5286,-22.7915
6,COQUIMBO,1915.4263,2176.9260,261.4996
7,COYHAIQUE,408.8663,401.3564,-7.5099
8,CURAUMA,4795.0065,6042.2150,1247.2086
9,ILLAPEL,376.2956,100.1641,-276.1314


In [76]:
wholeAggregated.to_excel('tabla agregada CD agrupacion variabilidad 92.5.xlsx')

In [77]:
stats_no_agrupados

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.4005,0.7071,1.7653,0.6717,1.0722,0.6265
1,515,ARICA,0.0419,0.1407,3.3622,0.1337,0.1756,0.7616
2,515,CALAMA,0.2196,0.4439,2.0217,0.4218,0.6413,0.6576
3,515,CASTRO,0.1207,0.3007,2.4909,0.2857,0.4064,0.7029
4,515,CHILLAN,0.2766,0.5116,1.8497,0.4860,0.7626,0.6373
...,...,...,...,...,...,...,...,...
23007,875225,VALDIVIA,0.0003,0.0041,14.6616,0.0039,0.0042,0.9330
23008,875359,SANTIAGO SUR,0.0084,0.2427,28.8444,0.2305,0.2390,0.9648
23009,875365,CURAUMA,0.0168,0.3430,20.3838,0.3258,0.3427,0.9509
23010,875365,SANTIAGO SUR,0.0084,0.2427,28.8444,0.2305,0.2390,0.9648


In [79]:
stats_no_agrupados[['ID_SKU_VENTA', 'DESCR_CENDIST', 'STD']].to_excel('stats.xlsx')